In [1]:
import numpy as np
import pandas as pd
import keras
import warnings
warnings.filterwarnings("ignore")

Using TensorFlow backend.


In [2]:
df= pd.read_csv("/home/samridhi/boatgang/dataset/data.csv", header=None, skiprows=1)
df.head()

,0,1
0,TCCGATATGTACCCGGACGTAATCCGCAGAATAAAGTATCTAAAAG...,1
1,GGTAACGAAGTGGTTACATTATGCTCTTCCGTTTCGGATTGTGAAC...,1
2,GCTTCGTCACGGACGTCCCTCGATTAGACTCGATATGTACCTAACG...,1
3,ATGCTTCTACACGCATTATACAGCCCCGCGTTTTAGTCCTTTTCAG...,1
4,GGACTAGTTGAGGAACAATGAACTCTTTTCAATTGAAACCGAGTTA...,1


In [3]:
df.shape

(60000, 2)

In [4]:
temp_sequences=df[0]
temp_target=df[1]

In [5]:
dict_i = {"A" : 0, "C": 1, "G": 2, "T" : 3}

In [6]:
def one_hot_encode(s):
    i=0
    ohe = np.zeros((len(s), 4))
    for k in s:
        ohe[i,dict_i[k]] = 1
        i+=1
    return ohe

In [7]:
ohe_sequences = np.array([one_hot_encode(s) for s in temp_sequences])
print ohe_sequences
ohe_sequences.shape

[[[0. 0. 0. 1.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  ...
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]
  [0. 0. 1. 0.]]

 [[0. 0. 1. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]
  ...
  [0. 1. 0. 0.]
  [1. 0. 0. 0.]
  [0. 1. 0. 0.]]

 [[0. 0. 1. 0.]
  [0. 1. 0. 0.]
  [0. 0. 0. 1.]
  ...
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]]

 ...

 [[0. 0. 0. 1.]
  [0. 0. 1. 0.]
  [0. 1. 0. 0.]
  ...
  [1. 0. 0. 0.]
  [1. 0. 0. 0.]
  [0. 0. 0. 1.]]

 [[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 1. 0. 0.]
  ...
  [1. 0. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 1. 0.]]

 [[0. 1. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  ...
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]
  [0. 0. 0. 1.]]]


(60000, 200, 4)

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Dropout,Flatten
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

We used the Keras deep learning framework (https://github.com/keras-team/keras) to train a CNN to classify
the sequences. The sequences were represented using a one-hot encoding with 4 channels (A, C, G and T). The
CNN architecture is as follows: Layer 1 is a convolutional layer with 40 filters of size 19 and ReLU activation
operating on one-hot encoded input sequences. Layer 2 is a max pooling layer of pool length 10. Layer 3 is a
fully connected layer of size 200 with dropout (p=0.5) and ReLU activation. Layer 4 is a fully connected layer
with a sigmoid activation. The model was trained with the Adam optimizer and binary cross-entropy loss until
no improvement was seen for 3 epochs on the validation set. The datasets, code and model are available at

In [9]:
model = Sequential()
model.add(Conv1D(filters=40, kernel_size=10, activation='relu', input_shape=(200,4)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 191, 40)           1640      
Total params: 1,640
Trainable params: 1,640
Non-trainable params: 0
_________________________________________________________________


In [10]:
model.add(MaxPooling1D(pool_size=10, strides=10))
model.add(Flatten())

In [11]:
model.add(Dense(200,  activation='relu'))

In [12]:
model.add(Dense(1, activation = "sigmoid"))

In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 191, 40)           1640      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 19, 40)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 760)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               152200    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 154,041
Trainable params: 154,041
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [15]:
data_train, data_test, labels_train, labels_test = train_test_split(ohe_sequences, temp_target, test_size=0.30, random_state=42, shuffle=True)

In [16]:
model.fit(data_train, labels_train, epochs=10, verbose=1)

Epoch 1/10
42000/42000 [==============================] - 9s 208us/step - loss: 0.2258 - acc: 0.8805
Epoch 2/10
42000/42000 [==============================] - 4s 84us/step - loss: 0.0046 - acc: 0.9991
Epoch 3/10
42000/42000 [==============================] - 3s 78us/step - loss: 0.0019 - acc: 0.9996
Epoch 4/10
42000/42000 [==============================] - 3s 80us/step - loss: 7.6755e-04 - acc: 0.9999
Epoch 5/10
42000/42000 [==============================] - 3s 78us/step - loss: 1.2692e-04 - acc: 1.0000
Epoch 6/10
42000/42000 [==============================] - 3s 83us/step - loss: 2.9053e-05 - acc: 1.0000
Epoch 7/10
42000/42000 [==============================] - 3s 77us/step - loss: 1.3797e-05 - acc: 1.0000
Epoch 8/10
42000/42000 [==============================] - 3s 78us/step - loss: 7.5737e-06 - acc: 1.0000
Epoch 9/10
42000/42000 [==============================] - 3s 79us/step - loss: 0.0015 - acc: 0.9995
Epoch 10/10
42000/42000 [==============================] - 3s 83us/step - loss:

In [17]:
scores = model.evaluate(data_test, labels_test)
print ("Test loss ", scores[0])
print ("Test acc ", scores[1])

18000/18000 [==============================] - 1s 35us/step
('Test loss ', 0.003057871212526354)
('Test acc ', 0.9995555555555555)


In [18]:
model.get_config()

{'layers': [{'class_name': 'Conv1D',
   'config': {'activation': 'relu',
    'activity_regularizer': None,
    'batch_input_shape': (None, 200, 4),
    'bias_constraint': None,
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'bias_regularizer': None,
    'data_format': 'channels_last',
    'dilation_rate': (1,),
    'dtype': 'float32',
    'filters': 40,
    'kernel_constraint': None,
    'kernel_initializer': {'class_name': 'VarianceScaling',
     'config': {'distribution': 'uniform',
      'mode': 'fan_avg',
      'scale': 1.0,
      'seed': None}},
    'kernel_regularizer': None,
    'kernel_size': (10,),
    'name': 'conv1d_1',
    'padding': 'valid',
    'strides': (1,),
    'trainable': True,
    'use_bias': True}},
  {'class_name': 'MaxPooling1D',
   'config': {'data_format': 'channels_last',
    'name': 'max_pooling1d_1',
    'padding': 'valid',
    'pool_size': (10,),
    'strides': (10,),
    'trainable': True}},
  {'class_name': 'Flatten',
   'config': {'d

In [19]:
import deeplift
import json
from deeplift.conversion import kerasapi_conversion

In [20]:
model.save_weights("my_weights.h5")

In [21]:
my_json_string = model.to_json()

In [22]:
def convertKerasJSONtoDeepLIFT(kerasJSON_str):
    jsonData = json.loads(kerasJSON_str)
    layersData = jsonData["config"]["layers"]
    jsonData["config"] = layersData
    return json.dumps(jsonData)

In [23]:
my_json_string = convertKerasJSONtoDeepLIFT(my_json_string)
with open("model.json", "w") as json_file:
    json_file.write(my_json_string)
keras_model_weights = "my_weights.h5"
keras_model_json = "model.json"

In [24]:
from keras.models import model_from_json
keras_model = model_from_json(open(keras_model_json).read())
keras_model.load_weights(keras_model_weights)

In [25]:
from deeplift.layers import NonlinearMxtsMode
import deeplift.conversion.kerasapi_conversion as kc
reload(deeplift.layers)
reload(deeplift.conversion.kerasapi_conversion)
from collections import OrderedDict

In [26]:
method_to_model = OrderedDict()
for method_name, nonlinear_mxts_mode in [
    #The genomics default = rescale on conv layers, revealcance on fully-connected
    ('rescale_conv_revealcancel_fc', NonlinearMxtsMode.DeepLIFT_GenomicsDefault),
    ('rescale_all_layers', NonlinearMxtsMode.Rescale),
    ('revealcancel_all_layers', NonlinearMxtsMode.RevealCancel),
    ('grad_times_inp', NonlinearMxtsMode.Gradient),
    ('guided_backprop', NonlinearMxtsMode.GuidedBackprop)]:
    method_to_model[method_name] = kc.convert_model_from_saved_files(
        h5_file=keras_model_weights,
        json_file=keras_model_json,
        nonlinear_mxts_mode=nonlinear_mxts_mode)

nonlinear_mxts_mode is set to: DeepLIFT_GenomicsDefault
For layer 0 the preceding linear layer is preact_0 of type Conv1D;
In accordance with nonlinear_mxts_mode=DeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to Rescale
Heads-up: current implementation assumes maxpool layer is followed by a linear transformation (conv/dense layer)
For layer 3 the preceding linear layer is preact_3 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
Heads-up: I assume sigmoid is the output layer, not an intermediate one; if it's an intermediate layer then please bug me and I will implement the grad func
For layer 4 the preceding linear layer is preact_4 of type Dense;
In accordance with nonlinear_mxts_modeDeepLIFT_GenomicsDefault we are setting the NonlinearMxtsMode to RevealCancel
nonlinear_mxts_mode is set to: Rescale
Heads-up: current implementation assumes maxpool layer is followed by a linear transformatio

In [27]:
from deeplift.util import compile_func
model_to_test = method_to_model['rescale_conv_revealcancel_fc']
deeplift_prediction_func = compile_func([model_to_test.get_layers()[0].get_activation_vars()],
                                         model_to_test.get_layers()[-1].get_activation_vars())
original_model_predictions = keras_model.predict(ohe_sequences, batch_size=200)
converted_model_predictions = deeplift.util.run_function_in_batches(
                                input_data_list=[ohe_sequences],
                                func=deeplift_prediction_func,
                                batch_size=200,
                                progress_update=None)
print("maximum difference in predictions:",np.max(np.array(converted_model_predictions)-np.array(original_model_predictions)))
assert np.max(np.array(converted_model_predictions)-np.array(original_model_predictions)) < 10**-5
predictions = converted_model_predictions

('maximum difference in predictions:', 0.0)


In [28]:
print("Compiling scoring functions")
method_to_scoring_func = OrderedDict()
for method,model in method_to_model.items():
    print("Compiling scoring function for: "+method)
    method_to_scoring_func[method] = model.get_target_contribs_func(find_scores_layer_idx=0,
                                                                    target_layer_idx=-2)
    
#To get a function that just gives the gradients, we use the multipliers of the Gradient model
gradient_func = method_to_model['grad_times_inp'].get_target_multipliers_func(find_scores_layer_idx=0,
                                                                              target_layer_idx=-2)
print("Compiling integrated gradients scoring functions")
integrated_gradients10_func = deeplift.util.get_integrated_gradients_function(
    gradient_computation_function = gradient_func,
    num_intervals=10)
method_to_scoring_func['integrated_gradients10'] = integrated_gradients10_func

Compiling scoring functions
Compiling scoring function for: rescale_conv_revealcancel_fc
Compiling scoring function for: rescale_all_layers
Compiling scoring function for: revealcancel_all_layers
Compiling scoring function for: grad_times_inp
Compiling scoring function for: guided_backprop
Compiling integrated gradients scoring functions


In [ ]:
background    = OrderedDict([('A', 0.25), ('C', 0.25), ('G', 0.25), ('T', 0.25)])

from collections import OrderedDict
method_to_task_to_scores = OrderedDict()
for method_name, score_func in method_to_scoring_func.items():
    print("on method",method_name)
    method_to_task_to_scores[method_name] = OrderedDict()
    for task_idx in [0,1,2]:
        scores = np.array(score_func(
                    task_idx=task_idx,
                    input_data_list=[ohe_sequences],
                    input_references_list=[
                     np.array([background['A'],
                               background['C'],
                               background['G'],
                               background['T']])[None,None,:]],
                    batch_size=200,
                    progress_update=None))
        assert scores.shape[2]==4
        #The sum over the ACGT axis in the code below is important! Recall that DeepLIFT
        # assigns contributions based on difference-from-reference; if
        # a position is [1,0,0,0] (i.e. 'A') in the actual sequence and [0.3, 0.2, 0.2, 0.3]
        # in the reference, importance will be assigned to the difference (1-0.3)
        # in the 'A' channel, (0-0.2) in the 'C' channel,
        # (0-0.2) in the G channel, and (0-0.3) in the T channel. You want to take the importance
        # on all four channels and sum them up, so that at visualization-time you can project the
        # total importance over all four channels onto the base that is actually present (i.e. the 'A'). If you
        # don't do this, your visualization will look very confusing as multiple bases will be highlighted at
        # every position and you won't know which base is the one that is actually present in the sequence!
        scores = np.sum(scores, axis=2)
        method_to_task_to_scores[method_name][task_idx] = scores

('on method', 'rescale_conv_revealcancel_fc')
('on method', 'rescale_all_layers')
('on method', 'revealcancel_all_layers')
('on method', 'grad_times_inp')
('on method', 'guided_backprop')
('on method', 'integrated_gradients10')
